## Adquisición y Simulación de Datos de Tráfico Web

In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.probability import FreqDist

import itertools
import string

In [2]:
# Establece la semilla para garantizar la reproducibilidad de los datos simulados.np.random.seed(42)

# Definición de las estrategias de contenido temático (muestra de artículos).
titulos = [
    "Estrategias de Contenido de Alto Impacto para Agencias Creativas",
    "La Tasa de Rebote Explicada: Cómo Afecta a tu SEO de Marketing",
    "API de Google Analytics: Cómo Conectarse con Python y Pandas",
    "5 Errores Comunes de Publicidad en Redes Sociales",
    "Guía Completa de la Optimización de Contenido para Vender más"
] * 20

# Creación del DataFrame que simula la respuesta de una API de Analítica Web.
data = pd.DataFrame({
    'ID_Contenido': range(1, 101),
    'URL': [f'/blog/articulo-{i}' for i in range(1, 101)],
    'Titulo': np.random.choice(titulos, 100),
    'Tiempo_en_Pagina': np.random.randint(30, 300, 100), 
    'Tasa_de_Rebote': np.random.uniform(0.20, 0.80, 100).round(2),
    'Visitas': np.random.randint(500, 15000, 100)
})

# Verificación inicial de la estructura del DataFrame.
print(data.head())

   ID_Contenido               URL  \
0             1  /blog/articulo-1   
1             2  /blog/articulo-2   
2             3  /blog/articulo-3   
3             4  /blog/articulo-4   
4             5  /blog/articulo-5   

                                              Titulo  Tiempo_en_Pagina  \
0  5 Errores Comunes de Publicidad en Redes Sociales                43   
1  API de Google Analytics: Cómo Conectarse con P...               192   
2  API de Google Analytics: Cómo Conectarse con P...                70   
3  5 Errores Comunes de Publicidad en Redes Sociales               143   
4  La Tasa de Rebote Explicada: Cómo Afecta a tu ...                84   

   Tasa_de_Rebote  Visitas  
0            0.39    13512  
1            0.30    12009  
2            0.58    12072  
3            0.71     5917  
4            0.41     2278  


In [3]:
# Muestra un resumen estadístico de las columnas numéricas
print(data.describe().round(2))

       ID_Contenido  Tiempo_en_Pagina  Tasa_de_Rebote   Visitas
count        100.00            100.00          100.00    100.00
mean          50.50            160.31            0.51   7701.96
std           29.01             74.52            0.18   3985.44
min            1.00             30.00            0.20    638.00
25%           25.75             93.50            0.35   4242.75
50%           50.50            154.00            0.56   8255.00
75%           75.25            224.25            0.65  10986.00
max          100.00            296.00            0.79  14969.00


In [4]:
# --- Segmentación de Alto Impacto ---

# Calcula los promedios dinámicamente para el benchmark.
tiempo_promedio_dinamico = data['Tiempo_en_Pagina'].mean()
tasa_rebote_promedio_dinamico = data['Tasa_de_Rebote'].mean()

# Filtra el contenido que supera el rendimiento promedio:
# (Tiempo en Página > Promedio) Y (Tasa de Rebote < Promedio).
alto_impacto = data[
    (data['Tiempo_en_Pagina'] > tiempo_promedio_dinamico) & 
    (data['Tasa_de_Rebote'] < tasa_rebote_promedio_dinamico)
].copy()

print(f"Artículos de Alto Impacto encontrados: {len(alto_impacto)} de {len(data)}")

# --- Análisis de Rendimiento Estratégico (Agrupación) ---

# Agrupa los datos originales por el título temático y calcula las métricas promedio.
analisis_por_titulo = data.groupby('Titulo').agg({
    'Visitas': 'sum',
    'Tiempo_en_Pagina': 'mean',
    'Tasa_de_Rebote': 'mean',
    'ID_Contenido': 'count'
}).sort_values(by='Tiempo_en_Pagina', ascending=False).round(2)

# Renombra la columna de conteo para claridad
analisis_por_titulo = analisis_por_titulo.rename(columns={'ID_Contenido': 'Num_Articulos'})

print("\nAnálisis de Rendimiento Promedio por Título Temático:")
analisis_por_titulo

Artículos de Alto Impacto encontrados: 23 de 100

Análisis de Rendimiento Promedio por Título Temático:


,Visitas,Tiempo_en_Pagina,Tasa_de_Rebote,Num_Articulos
Titulo,,,,
La Tasa de Rebote Explicada: Cómo Afecta a tu SEO de Marketing,158500,172.14,0.50,21
5 Errores Comunes de Publicidad en Redes Sociales,152757,170.15,0.49,20
Estrategias de Contenido de Alto Impacto para Agencias Creativas,119381,165.94,0.48,16
Guía Completa de la Optimización de Contenido para Vender más,179889,150.82,0.54,22
API de Google Analytics: Cómo Conectarse con Python y Pandas,159669,144.76,0.53,21


In [5]:
# Inicializa el tokenizador alternativo (soluciona conflicto en algunos entornos NLTK)
tokenizer = TreebankWordTokenizer()

# 1. Definición de recursos de limpieza
spanish_stopwords = set(stopwords.words('spanish')) 
puntuacion = string.punctuation

# 2. Función para estandarizar el texto y extraer tokens de valor
def limpiar_y_tokenizar(titulo):
    # a. Tokenizar el texto en minúsculas
    tokens = tokenizer.tokenize(titulo.lower())
    
    # b. Eliminar stopwords y tokens no alfanuméricos
    palabras_limpias = [
        palabra for palabra in tokens 
        if palabra.isalnum() and palabra not in spanish_stopwords and palabra not in puntuacion
    ]
    return palabras_limpias

# 3. Aplicación al DataFrame: Genera la columna con las palabras clave limpias
alto_impacto['Palabras_Clave'] = alto_impacto['Titulo'].apply(limpiar_y_tokenizar)

print("Palabras Clave limpias generadas con NLTK. Listo para el conteo de frecuencias.")
alto_impacto[['Titulo', 'Palabras_Clave']].head()

Palabras Clave limpias generadas con NLTK. Listo para el conteo de frecuencias.


,Titulo,Palabras_Clave
1,API de Google Analytics: Cómo Conectarse con P...,"[api, google, analytics, cómo, conectarse, pyt..."
6,Guía Completa de la Optimización de Contenido ...,"[guía, completa, optimización, contenido, vender]"
12,Estrategias de Contenido de Alto Impacto para ...,"[estrategias, contenido, alto, impacto, agenci..."
20,Estrategias de Contenido de Alto Impacto para ...,"[estrategias, contenido, alto, impacto, agenci..."
22,Estrategias de Contenido de Alto Impacto para ...,"[estrategias, contenido, alto, impacto, agenci..."


In [6]:
# --- Análisis de Frecuencia de Palabras Clave ---

# 1. Aplanar la lista: Convierte la columna de listas (Palabras_Clave) en una sola lista gigante.
lista_plana_palabras = list(itertools.chain.from_iterable(alto_impacto['Palabras_Clave']))

# 2. Calcular la Distribución de Frecuencias
frecuencia_palabras = FreqDist(lista_plana_palabras)

# 3. Mostrar las 15 Palabras Clave más Comunes (Insight Táctico)
print("Top 15 Palabras Clave asociadas al Contenido de Alto Impacto:")
print("-" * 40)
for palabra, frecuencia in frecuencia_palabras.most_common(15):
    print(f"- {palabra}: {frecuencia} veces")

Top 15 Palabras Clave asociadas al Contenido de Alto Impacto:
----------------------------------------
- cómo: 10 veces
- contenido: 9 veces
- tasa: 7 veces
- rebote: 7 veces
- explicada: 7 veces
- afecta: 7 veces
- seo: 7 veces
- marketing: 7 veces
- estrategias: 5 veces
- alto: 5 veces
- impacto: 5 veces
- agencias: 5 veces
- creativas: 5 veces
- guía: 4 veces
- completa: 4 veces
